In [2]:
test_mode = False

In [3]:
import pandas as pd
import hopsworks
from utils import *
import json
import os
import warnings
from dotenv import load_dotenv
import datetime
#from datetime import timezone

warnings.filterwarnings("ignore")

/opt/anaconda3/envs/crypto-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")


project = hopsworks.login()

2025-01-01 21:23:41,341 INFO: Initializing external client
2025-01-01 21:23:41,341 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-01 21:23:42,874 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164448


In [5]:

fs = project.get_feature_store() 
#secrets = secrets_api(project.name)

# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60


today


1735759424

In [6]:

# Step 2: Retrieve the feature group by name and version
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

# Step 3: Read the data from the feature group
s_data_df = solana_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
solana_max_date = s_data_df['timestamp'].max()
solana_difference = (today - solana_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {solana_max_date}")
print(f"Difference in days: {solana_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.65s) 
The maximum date is: 1735344000
Difference in days: 4


In [7]:
# Step 2: Retrieve the feature group by name and version
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)

# Step 3: Read the data from the feature group
b_data_df = bitcoin_feature_group.select(["timestamp"]).read()

# Step 4: Find the maximum date
bitcoin_max_date = b_data_df['timestamp'].max()
bitcoin_difference = (today - bitcoin_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {bitcoin_max_date}")
print(f"Difference in days: {bitcoin_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 
The maximum date is: 1735344000
Difference in days: 4


In [8]:
#fs = project.get_feature_store(name='kth_lab1_2024_featurestore')
fs = project.get_feature_store()

fng_feature_group = fs.get_feature_group(
    name='f_n_g_index', 
    version=8)
fng_df = fng_feature_group.read()

# Step 4: Find the maximum date
fng_max_date = int(fng_df['timestamp'].max())
fng_difference = (today - fng_max_date)//seconds_in_one_day

# Step 5: Print the result
print(f"The maximum date is: {fng_max_date}")
print(f"Difference in days: {fng_difference}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.75s) 
The maximum date is: 1735344000
Difference in days: 4


### Getting missing data

### Request solana price from last available data till now

In [9]:

url = "https://data-api.cryptocompare.com//spot/v1/historical/days"

# Load environment variables from the .env file
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "SOL-USD",  
    "limit": solana_difference,  
    "api_key": crypto_api_key,
}
if solana_difference!=0:
    sol_response = trigger_request(url, params)
    sol_response_df = pd.DataFrame(sol_response["Data"])
    sol_response_df = sol_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    sol_response_df.columns = sol_response_df.columns.str.lower()
    print("Num entries to add: ", len(sol_response_df))
else:
    sol_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

Num entries to add:  4


In [10]:
sol_response_df.head()

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735430400,195.15,197.19,188.26,189.60,120179.830193,45590.791614,74589.038579
1,1735516800,189.60,196.10,185.58,191.06,186701.859563,80979.111912,105722.747651
2,1735603200,191.06,198.73,187.76,188.95,169485.435743,98813.418835,70672.016908
3,1735689600,188.95,192.19,187.63,191.97,68217.177694,38111.182699,30105.994995


### Request bitcoin price from last available data till now

In [11]:
load_dotenv()
crypto_api_key = os.getenv("CRYPTO_API_KEY")


params = {
    "market":"kraken",
    "instrument": "BTC-USD",  
    "limit": bitcoin_difference,  
    "api_key": crypto_api_key,
}

if bitcoin_difference>0:
    bitcoin_response = trigger_request(url, params)
    bitcoin_response_df = pd.DataFrame(bitcoin_response["Data"])
    bitcoin_response_df = bitcoin_response_df[["TIMESTAMP", 'OPEN', 'HIGH', 'LOW', 'CLOSE', "VOLUME", 'VOLUME_BUY', 'VOLUME_SELL']]
    bitcoin_response_df.columns = bitcoin_response_df.columns.str.lower()
    print("Num entries to add: ", len(bitcoin_response_df))
else:
    bitcoin_response_df = pd.DataFrame(columns=["timestamp","open","high", "low","close", "volume", "volume_buy", "volume_sell"])

bitcoin_response_df.head()

Num entries to add:  4


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735430400,95204.1,95204.1,92860.7,93551.0,698.547127,229.720452,468.826675
1,1735516800,93551.0,94858.7,91256.6,92611.3,2168.291728,994.214537,1174.077191
2,1735603200,92611.3,96040.0,91934.1,93370.9,1697.950414,943.514778,754.435636
3,1735689600,93370.9,94250.0,92588.1,93999.9,435.492907,213.283399,222.209508


In [12]:
bitcoin_response_df

,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1735430400,95204.1,95204.1,92860.7,93551.0,698.547127,229.720452,468.826675
1,1735516800,93551.0,94858.7,91256.6,92611.3,2168.291728,994.214537,1174.077191
2,1735603200,92611.3,96040.0,91934.1,93370.9,1697.950414,943.514778,754.435636
3,1735689600,93370.9,94250.0,92588.1,93999.9,435.492907,213.283399,222.209508


### Request fear and greed index values from last available data till now

In [23]:
url = "https://api.alternative.me/fng/?"

# Set the parameters for the API request
params = {
    "limit": fng_difference  # Number of days of data to retrieve
}

if fng_difference>0:
    fng_response = trigger_request(url, params)
    fng_df = pd.DataFrame(fng_response['data'])
    fng_df['timestamp'] = fng_df['timestamp'].astype(float)
    fng_df.rename(columns={'value_classification': 'fng_classification'}, inplace=True)
    fng_df.rename(columns={'value': 'fng_value'}, inplace=True)

    fng_df.drop(columns=["time_until_update"],inplace=True)
    fng_df.head()
else:
    fng_df = pd.DataFrame(columns=["fng_value","fng_classification","timestamp"])
fng_df.head()

,fng_value,fng_classification,timestamp
0,66,Greed,1.735690e+09
1,64,Greed,1.735603e+09
2,65,Greed,1.735517e+09
3,73,Greed,1.735430e+09


### Now we need to use the responses to update the data in Hopsworks

In [19]:
if not test_mode and sol_response_df.shape[0]>0:
    solana_feature_group.insert(sol_response_df)



Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164448/jobs/named/solana_1_offline_fg_materialization/executions


In [20]:
if not test_mode and bitcoin_response_df.shape[0]>0:
    bitcoin_feature_group.insert(bitcoin_response_df)


Uploading Dataframe: 25.00% |██▌       | Rows 1/4 | Elapsed Time: 00:01 | Remaining Time: 00:04

KeyboardInterrupt: 

In [21]:
fng_df.head()

,fng_value,fng_classification,timestamp
0,66,Greed,1735689600
1,64,Greed,1735603200
2,65,Greed,1735516800
3,73,Greed,1735430400


In [24]:
if not test_mode and fng_df.shape[0]>0:
    fng_feature_group.insert(fng_df)

Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: f_n_g_index_8_offline_fg_materialization


%6|1735759616.707|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 54882ms in state UP)
%6|1735759617.221|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50382ms in state UP)


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164448/jobs/named/f_n_g_index_8_offline_fg_materialization/executions


%6|1735759667.399|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 100573ms in state UP)
%6|1735759717.965|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50030ms in state UP, 1 identical error(s) suppressed)
%6|1735759717.973|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50000ms in state UP, 1 identical error(s) suppressed)
%6|1735759768.522|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50001ms in state UP, 1 identical error(s) suppressed)
%6|1735759768.628|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50132ms in state UP, 1 identical error(s) suppressed)
%6|1735759819.567|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://